# Product Screening

This notebook screens products.

In [2]:
import duckdb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
db = duckdb.connect()

In [4]:
%load_ext sql
%sql db --alias duckdb

Tip: You may define configurations in /pit/trec/trec-product-search-recs/pyproject.toml or /home/mde48/.jupysql/config.

Did not find user configurations in /pit/trec/trec-product-search-recs/pyproject.toml.

In [5]:
%%sql
ATTACH 'ucsd-products.duckdb' AS rd (READ_ONLY);

Running query in 'duckdb'

Success


## Initial Statistics

In [6]:
%%sql
SELECT category, COUNT(asin) AS n_products,
    MEDIAN(n_desc_entries) AS dc, 
    MEDIAN(COALESCE(desc_len, 0)) AS dlen,
    MEDIAN(rating_count) AS med_ratings
FROM rd.products
GROUP BY category

Running query in 'duckdb'

category,n_products,dc,dlen,med_ratings
Software,89251,1.0,603.0,12.0
All_Beauty,112590,0.0,0.0,9.0
Baby_Products,217724,1.0,21.0,19.0
Unknown,390006,0.0,0.0,3.0
Industrial_and_Scientific,427564,1.0,68.0,8.0
Appliances,94327,1.0,25.0,13.0
Patio_Lawn_and_Garden,851907,1.0,53.0,14.0
Subscription_Boxes,641,0.0,0.0,42.0
Toys_and_Games,890874,1.0,53.0,20.0
Movies_and_TV,748224,1.0,0.0,60.0


## Viable Categories

Mark some categories as *conceptually* viable.

In [7]:
%%sql
CREATE OR REPLACE TABLE face_cats (
    category VARCHAR NOT NULL
)

Running query in 'duckdb'

Count


In [8]:
face_cats = [
    'Electronics',
    'Health_and_Household',
    'Home_and_Kitchen',
    'Cell_Phones_and_Accessories',
    'Office_Products',
    'Patio_Lawn_and_Garden',
    'Sports_and_Outdoors',
    'Tools_and_Home_Improvement',
]
for cat in face_cats:
    db.execute('INSERT INTO face_cats VALUES (?)', [cat])

## Initial Filtering

In [19]:
%%sql
CREATE OR REPLACE VIEW maybe_products AS
SELECT prod_id, asin
FROM rd.products
JOIN face_cats USING (category)
WHERE asin IN (
    SELECT asin FROM rd.esci_products
    -- UNION DISTINCT
    -- SELECT asin FROM rd.m2_products
)
AND rating_count >= 10
AND desc_len > 50

Running query in 'duckdb'

Count


In [20]:
%%sql
SELECT category, COUNT(p.asin) AS n_products,
    MEDIAN(COALESCE(desc_len, 0)) AS desc_len,
    MEDIAN(rating_count) AS med_ratings
FROM rd.products p
JOIN maybe_products USING (prod_id)
GROUP BY category
ORDER BY n_products DESC

Running query in 'duckdb'

category,n_products,desc_len,med_ratings
Home_and_Kitchen,23386,387.0,204.0
Electronics,12938,449.0,209.0
Sports_and_Outdoors,11613,384.0,181.0
Health_and_Household,10831,412.0,517.0
Tools_and_Home_Improvement,10453,402.0,184.0
Patio_Lawn_and_Garden,6919,424.0,219.0
Office_Products,6302,361.0,243.0
Cell_Phones_and_Accessories,3056,360.0,303.0


## Add Co-occurrance Filters

Now we're going to use the 5-core to see how many co-occurrances items in this set have.

In [21]:
ic_queries = ' UNION ALL '.join(
    f"""
    SELECT user_id, parent_asin AS asin, rating, timestamp
    FROM 'ucsd-2023/5core/{cat}.csv.gz'
    """
    for cat in face_cats
)
db.execute(f'CREATE OR REPLACE VIEW item_rates AS {ic_queries}')

In [22]:
%%sql
CREATE OR REPLACE TABLE item_corates AS
SELECT mp1.prod_id AS prod1, mp2.prod_id AS prod2, COUNT(*) AS n_corates
FROM item_rates r1
JOIN maybe_products mp1 ON (mp1.asin = r1.asin)
JOIN item_rates r2 USING (user_id)
JOIN maybe_products mp2 ON (mp2.asin = r2.asin)
WHERE mp1.prod_id != mp2.prod_id
GROUP BY mp1.prod_id, mp2.prod_id

Running query in 'duckdb'

Count
7909376


In [23]:
%%sql
SELECT category, COUNT(p.asin) AS n_products,
    MEDIAN(COALESCE(desc_len, 0)) AS desc_len,
    MEDIAN(rating_count) AS med_ratings,
    MEDIAN(n_corates) AS med_corated,
FROM rd.products p
JOIN maybe_products USING (prod_id)
LEFT JOIN (
    SELECT prod1 AS prod_id, COUNT(prod2) AS n_corates
    FROM item_corates
    GROUP BY prod1
) USING (prod_id)
GROUP BY category
ORDER BY n_products DESC

Running query in 'duckdb'

category,n_products,desc_len,med_ratings,med_corated
Home_and_Kitchen,23386,387.0,204.0,35.0
Electronics,12938,449.0,209.0,49.0
Sports_and_Outdoors,11613,384.0,181.0,40.0
Health_and_Household,10831,412.0,517.0,80.0
Tools_and_Home_Improvement,10453,402.0,184.0,53.0
Patio_Lawn_and_Garden,6919,424.0,219.0,61.0
Office_Products,6302,361.0,243.0,59.0
Cell_Phones_and_Accessories,3056,360.0,303.0,34.0
